In [16]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import html5lib
import time

In [19]:

def fetch_chotot_data(pages=3, limit=10): #Hà Đông, Hà Nội
    url = "https://gateway.chotot.com/v1/public/ad-listing" # link bất động sản Hà Đông

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) " # Trình duyệt giả lập
                      "AppleWebKit/537.36 (KHTML, like Gecko) "# Trình duyệt giả lập
                      "Chrome/140.0.0.0 Safari/537.36", 
        "Origin": "https://www.nhatot.com", # Trang gốc
        "Referer": "https://www.nhatot.com/"# Trang tham chiếu
    }

    results = [] # Danh sách kết quả

    for page in range(1, pages + 1):
        params = {
            "limit": limit,
            "page": page,
            "cg": "1000",            # Bất động sản
            "region_v2": "12000",    # Hà Nội
            "area_v2": "12086",      # Quận Hà Đông
            "st": "s,k",
            "key_param_included": "true",
            "video_count_included": "true"
        }

        r = requests.get(url, headers=headers, params=params, timeout=10)
        r.raise_for_status()
        data = r.json()
        
        ads = data.get("ads", [])
        if not ads: # Nếu không có tin nào thì bỏ qua
            print(f"Trang {page}: không có dữ liệu.")
            continue
        # lấy thông tin từng tin như diện tích, địa chỉ, giá, tiêu đề, hình ảnh
        for ad in ads:
            # diện tích thực tế
            area_real = None
            if "parameters" in ad:
                for p in ad["parameters"]:
                    if p.get("key") in ["size", "size_in_m2"]:
                        area_real = p.get("value")

            # địa chỉ fallback
            address = ad.get("address")
            if not address:
                address_parts = [
                    ad.get("ward_name"),
                    ad.get("area_name"),
                    ad.get("region_name")
                ]
                address = ", ".join([x for x in address_parts if x])

            results.append({
                "id": ad.get("list_id"),
                "title": ad.get("subject"),
                "price": ad.get("price_string"),
                "area_m2": area_real,
                "address": address,
                "images": ", ".join(ad.get("images", []))
            })

        
    
        print(f"✅ Lấy xong trang {page}, {len(ads)} tin.")
    
    return results


if __name__ == "__main__":
    data = fetch_chotot_data(pages=5, limit=20)
    print(f"\nTổng số tin lấy được: {len(data)}")

    df = pd.DataFrame(data)
    df.to_csv("nhatot_bds.csv", index=False, encoding="utf-8-sig")
    print("💾 Đã lưu dữ liệu vào nhatot_bds.csv")
    


✅ Lấy xong trang 1, 20 tin.
✅ Lấy xong trang 2, 20 tin.
✅ Lấy xong trang 3, 20 tin.
✅ Lấy xong trang 4, 20 tin.
✅ Lấy xong trang 5, 20 tin.

Tổng số tin lấy được: 100
💾 Đã lưu dữ liệu vào nhatot_bds.csv


In [12]:
data

[{'id': 127795417,
  'title': 'CC Cao Cấp, Tố Hữu – Hà Đông, 108 mét, mặt phố, 3 ngủ, Full NT, ở ngay',
  'price': '6,35 tỷ',
  'area_m2': None,
  'address': 'Phường La Khê, Quận Hà Đông, Hà Nội',
  'images': 'https://cdn.chotot.com/crPVLstcfr-aq_AR5eQoZMrtwZtvI6wBJ3kAvwyaEhY/preset:view/plain/f1e91299e785296ec007dc15684818f5-2949689810672991418.jpg, https://cdn.chotot.com/kkIe4YULe1zJw3Kwu1XGtHYg06S-bZ6d0WnGExaB3w4/preset:view/plain/d39584ac265d171513651e942a917896-2949689810953313813.jpg, https://cdn.chotot.com/b9xPnObPmElNqZo9piTD6UEYT3Fgp4x3qr-R-pAScWc/preset:view/plain/e9f5fcc81532dd8929a4cf53c5be274c-2949689810589626850.jpg, https://cdn.chotot.com/eOOV23ArGeStQ_S6ReI8_J9CAcZIkYe4BhevPZcd96M/preset:view/plain/31b23697c33ff3dbacfdaacd57a1bf55-2949689810698626044.jpg, https://cdn.chotot.com/9tbIknMDMnGkbRF1oAMStrGp7tPxnFOnquHuNIaqXWs/preset:view/plain/b44592ee50aa454bf90ae4967ef5ce23-2949689810495030506.jpg'},
 {'id': 127769917,
  'title': '🏡 CHỦ CẦN BÁN NHÀ PHÚ LƯƠNG – HÀ ĐÔNG',
  